# Install Libraries

In [4]:
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 58.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 63.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.3/110.3 kB 30.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 72.4 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 50.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.2/753.2 MB 4.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 29.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 20.1 MB/s eta 0:

# Modifications to match the experiment settings

In [5]:
!pwd

/workspace/ActiveLearning/AlphaMix/alpha_mix_active_learning


In [7]:
!ls

Img1.png   Run.ipynb	   dataset.py  main.py	query_strategies
README.md  agg_results.py  log	       models	requirements.txt


In [20]:
device =  torch.device('cuda:3')

In [21]:
import copy
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR
import torch.nn as nn
import numpy as np
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out1 = self.layer1(out)
        out2 = self.layer2(out1)
        out3 = self.layer3(out2)
        out4 = self.layer4(out3)
        out = F.avg_pool2d(out4, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out, [out1, out2, out3, out4]


def ResNet18(num_classes = 10):
    return ResNet(BasicBlock, [2,2,2,2], num_classes)

In [22]:
clf = ResNet18(num_classes=10).to(device)#원래 이렇게 오래 걸림?

In [28]:
clf.fc

AttributeError: 'ResNet' object has no attribute 'fc'

# Train Script

In [29]:
!python main.py \
        --data_name CIFAR10 --data_dir '/workspace/ActiveLearning/AlphaMix/alpha_mix_active_learning/' --log_dir '/workspace/ActiveLearning/AlphaMix/alpha_mix_active_learning/log/' \
        --n_init_lb 100 --n_query 100 --n_round 10 --learning_rate 0.1 --n_epoch 200 --model resnet18 \
        --strategy AlphaMixSampling --alpha_opt

Files already downloaded and verified
Files already downloaded and verified
number of labeled pool: 100
number of unlabeled pool: 49900
number of validation pool: 0
number of testing pool: 10000
on device: cuda:2
query strategy: AlphaMixSampling name: AlphaMixSampling
initializing...
model initialized
CIFAR10
SEED 1
AlphaMixSampling
SGD optimizer...
Training started...
  0%|                                                   | 0/200 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "main.py", line 515, in <module>
    supervised_learning(args)
  File "main.py", line 152, in supervised_learning
    al_train(args, train_args, train_params, args.strategy)
  File "main.py", line 180, in al_train
    al_train_sub_experiment(args, train_args, train_params, strategy_name, general_path, seed)
  File "main.py", line 305, in al_train_sub_experiment
    strategy.train(name='0')
  File "/workspace/ActiveLearning/AlphaMix/alpha_mix_active_learning/query_strategies/strategy.py", line 27, in 

In [31]:
!nvcc --version

/bin/sh: 1: nvcc: not found


In [ ]:
!python main.py \
        --data_name CIFAR10 --data_dir '/workspace/ActiveLearning/AlphaMix/alpha_mix_active_learning/' --log_dir '/workspace/ActiveLearning/AlphaMix/alpha_mix_active_learning/log/' \
        --n_init_lb 100 --n_query 100 --n_round 10 --learning_rate 0.1 --n_epoch 200 --model resnet18 \
        --strategy AlphaMixSampling --alpha_opt